In [1]:
import numpy as np
import pandas as pd

In [20]:
# Generate example data
date_range = pd.date_range(start="2020-01-01", periods=1_000, freq="d")
data = np.random.normal(50, 8, 1000)
df = pd.DataFrame(data, index=date_range, columns=["value"])

# Change the mean starting from 2022-01-21
new_mean = 65
df.loc["2022-01-01":, "value"] = np.random.normal(
    new_mean, 8, len(df.loc["2022-01-01":])
)

# Change the mean starting from 2022-01-21
new_mean = 30
df.loc["2022-08-01":, "value"] = np.random.normal(
    new_mean, 8, len(df.loc["2022-08-01":])
)

In [21]:
df.head()

,value
2020-01-01,56.777212
2020-01-02,48.721184
2020-01-03,50.171163
2020-01-04,49.048853
2020-01-05,55.090884


In [22]:
from spyc import SPC, x_chart, mr_chart, plotly_chart, xbar_chart

In [23]:
spc_example = SPC(data_in=df, target_col="value")

In [24]:
# OPTIONAL. i.e., new beds added and control lines need to be re-calculated
# (can be done once, or multiple times)

spc_example.add_process_change_date(
    change_date="2022-01-01",
    fix_control_start_dt="2022-01-01",
    fix_control_end_dt="2022-05-01",
    process_name="New ward",
)

spc_example.add_process_change_date(
    change_date="2022-08-01",
    fix_control_start_dt="2022-08-01",
    fix_control_end_dt="2022-09-01",
    process_name="New ward",
)

In [25]:
# OPTIONAL. Include seasonal variation. For example, CL calculated for each month.
# Add seasonality


# Need to define function that returns unique value for each seasonal period.
def month_seaonal(input_dates):
    return input_dates.month

In [26]:
# Pass in function to add_seasonality method.
spc_example.add_seasonality(season_func=month_seaonal)

In [27]:
x_chart_data = spc_example.calculate_spc(
    spc_calc_func=x_chart,  # Type of chart
    rule_1=True,
    rule_5=True,  # Rules to test
    min_data_req=15,
)  #  Min data required for each seasonal period
# (seasonality will be ignored if below threshold).
x_chart_data.head()

C:\Users\benjamin.holdsworth\Desktop\python-working\spyc\spyc\core.py:271: UserWarning:

Not enough data to build reliable estimate of mean for each season following 2022-01-01 00:00:00 change period. Global mean will be estimated until more data is added (control line calculation period is changed, or more data collected)

C:\Users\benjamin.holdsworth\Desktop\python-working\spyc\spyc\core.py:271: UserWarning:

Not enough data to build reliable estimate of mean for each season following 2022-08-01 00:00:00 change period. Global mean will be estimated until more data is added (control line calculation period is changed, or more data collected)



,CL,LCL,UCL,process,season,Rule 1 violation,Rule 5 violation
2020-01-01,48.355741,25.557598,71.153885,56.777212,1,0,0
2020-01-02,48.355741,25.557598,71.153885,48.721184,1,0,0
2020-01-03,48.355741,25.557598,71.153885,50.171163,1,0,0
2020-01-04,48.355741,25.557598,71.153885,49.048853,1,0,0
2020-01-05,48.355741,25.557598,71.153885,55.090884,1,0,0


In [28]:
# Visualise chart using Plotly
plotly_chart(x_chart_data)